#GUI for Brain_MRI_Diagnosis
###Using Gradio to Build Interface

In [ ]:
from huggingface_hub import hf_hub_download
import tensorflow as tf
import numpy as np
import gradio as gr
from PIL import Image
import plotly.graph_objects as go
from datetime import datetime
import json
model_path = hf_hub_download(
    repo_id="Shorya308/Brain_MRI_Scanner",
    filename="brain_tumor_model2.keras"
)

model = tf.keras.models.load_model(model_path, compile=False)


# Configuratio
IMG_SIZE = (256, 256)
CLASS_NAMES = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Enhanced medical information
MEDICAL_DATA = {
    "glioma": {
        "title": "Glioma",
        "full_name": "Glial Cell Tumor",
        "malignancy": "High (Grade II-IV)",
        "description": "Gliomas originate from glial cells that provide structural and metabolic support to neurons. They account for approximately 30% of all brain and central nervous system tumors and 80% of malignant brain tumors. Gliomas are classified into grades I-IV based on WHO criteria, with glioblastoma (Grade IV) being the most aggressive form.",
        "symptoms": [
            "Persistent morning headaches with nausea or vomiting",
            "New-onset seizures in adults without epilepsy history",
            "Progressive motor weakness or numbness in extremities",
            "Visual disturbances including blurred or double vision",
            "Speech and language difficulties (aphasia)",
            "Cognitive changes: memory loss, confusion, personality alterations",
            "Balance and coordination impairment",
            "Unexplained fatigue and drowsiness"
        ],
        "risk_factors": [
            "Ionizing radiation exposure",
            "Rare genetic syndromes (Li-Fraumeni, Neurofibromatosis)",
            "Family history of glioma (accounts for 5% of cases)",
            "Age: increased risk after 40 years"
        ],
        "diagnosis": [
            "MRI with gadolinium contrast (gold standard)",
            "CT scan for initial screening",
            "PET scan for tumor metabolic activity",
            "Stereotactic biopsy for histological confirmation",
            "Molecular markers: IDH1/2, MGMT, 1p/19q status"
        ],
        "treatments": [
            "Maximal safe surgical resection with neuro-navigation",
            "Adjuvant radiotherapy (54-60 Gy over 6 weeks)",
            "Temozolomide chemotherapy (Stupp protocol)",
            "Tumor Treating Fields (TTF/Optune device)",
            "Bevacizumab for recurrent high-grade gliomas",
            "Clinical trials: immunotherapy, CAR-T cells, oncolytic viruses",
            "Supportive care: anti-epileptics, corticosteroids, rehabilitation"
        ],
        "prognosis": "Grade II: 5-10 year survival. Grade III: 2-5 years. Glioblastoma (Grade IV): median survival 15-18 months with treatment. Early detection and molecular subtypes significantly impact outcomes.",
        "prevalence": "Incidence: 6 per 100,000 people annually. Median age at diagnosis: 64 years. Male:Female ratio approximately 1.5:1.",
        "color": "#FF1744",
        "icon": "⚠️"
    },
    "meningioma": {
        "title": "Meningioma",
        "full_name": "Meningeal Tumor",
        "malignancy": "Low (90% benign)",
        "description": "Meningiomas arise from arachnoid cap cells of the meninges, the protective membranes surrounding the brain and spinal cord. They represent the most common primary intracranial tumor. Approximately 90% are WHO Grade I (benign), 7% are Grade II (atypical), and 2-3% are Grade III (anaplastic/malignant).",
        "symptoms": [
            "Gradual onset headaches, often localized",
            "Seizures (in 20-50% of patients)",
            "Visual problems: blurred vision, double vision, visual field defects",
            "Hearing loss or tinnitus (acoustic meningiomas)",
            "Focal neurological deficits: weakness, numbness",
            "Cognitive changes: memory problems, concentration difficulties",
            "Anosmia (loss of smell) in olfactory groove meningiomas",
            "Personality or behavioral changes"
        ],
        "risk_factors": [
            "Female sex (2-3 times higher risk)",
            "Advanced age (peak incidence 60-70 years)",
            "Prior cranial radiation therapy",
            "Neurofibromatosis type 2 (NF2 gene mutation)",
            "Hormone-related factors (estrogen, progesterone)"
        ],
        "diagnosis": [
            "MRI with contrast (shows dural tail sign)",
            "CT scan (identifies calcifications and bone changes)",
            "Angiography for surgical planning",
            "Histopathological examination post-resection",
            "Molecular profiling for atypical/malignant variants"
        ],
        "treatments": [
            "Active surveillance for small, asymptomatic tumors",
            "Complete surgical resection (Simpson Grade I-II)",
            "Stereotactic radiosurgery (Gamma Knife, CyberKnife) for small tumors",
            "Fractionated radiotherapy for large or inoperable tumors",
            "Observation with serial MRI scans (every 6-12 months)",
            "Hormonal therapy (investigational)",
            "Chemotherapy for recurrent malignant meningiomas"
        ],
        "prognosis": "Grade I: >95% 5-year survival with complete resection. Grade II: 75-80% 5-year survival. Grade III: 50-60% 5-year survival. Recurrence rates: Grade I (7-20%), Grade II (30-40%), Grade III (50-80%).",
        "prevalence": "Most common primary brain tumor (36-40% of all cases). Incidence: 8-10 per 100,000 people annually. Female predominance especially pronounced in middle-aged women.",
        "color": "#FF9100",
        "icon": "🟡"
    },
    "pituitary": {
        "title": "Pituitary Adenoma",
        "full_name": "Pituitary Gland Tumor",
        "malignancy": "Very Low (>99% benign)",
        "description": "Pituitary adenomas are benign neoplasms of the anterior pituitary gland. They are classified as microadenomas (<10mm) or macroadenomas (≥10mm), and as functioning (hormone-secreting) or non-functioning. These tumors can cause problems through hormonal hypersecretion or mass effect on surrounding structures.",
        "symptoms": [
            "Bitemporal hemianopsia (tunnel vision from optic chiasm compression)",
            "Hyperprolactinemia: irregular menses, galactorrhea, infertility (women)",
            "Hypogonadism: erectile dysfunction, decreased libido (men)",
            "Acromegaly: enlarged hands, feet, facial features (GH excess)",
            "Cushing's disease: weight gain, moon face, striae (ACTH excess)",
            "Hyperthyroidism symptoms (TSH-secreting, rare)",
            "Hypopituitarism: fatigue, weakness, weight changes",
            "Headaches (often retro-orbital)"
        ],
        "risk_factors": [
            "Multiple Endocrine Neoplasia type 1 (MEN1)",
            "Family history (5% of cases)",
            "McCune-Albright syndrome",
            "Carney complex",
            "Age: 30-50 years most common"
        ],
        "diagnosis": [
            "Dedicated pituitary MRI with dynamic contrast",
            "Comprehensive hormonal panel (prolactin, GH, ACTH, TSH, IGF-1)",
            "Visual field testing (perimetry)",
            "Endocrine stimulation/suppression tests",
            "Petrosal sinus sampling for ACTH-dependent Cushing's"
        ],
        "treatments": [
            "Medical therapy: dopamine agonists (cabergoline, bromocriptine) for prolactinomas",
            "Transsphenoidal surgery (endoscopic endonasal approach)",
            "Somatostatin analogs for acromegaly (octreotide, lanreotide)",
            "Stereotactic radiosurgery for residual/recurrent tumors",
            "Conventional radiotherapy for large or aggressive tumors",
            "Hormone replacement therapy for hypopituitarism",
            "Lifelong endocrine monitoring and management"
        ],
        "prognosis": "Excellent for microadenomas with treatment. Prolactinomas: 80-90% remission with dopamine agonists. Surgical cure rates: microadenomas 70-90%, macroadenomas 40-60%. Very low mortality rate.",
        "prevalence": "Incidence: 4 per 100,000 annually. Found in 10-20% of routine autopsies (often clinically silent). Accounts for 10-15% of intracranial neoplasms. Slight female predominance.",
        "color": "#9C27B0",
        "icon": "🟣"
    },
    "notumor": {
        "title": "No Tumor Detected",
        "full_name": "Normal Brain Tissue",
        "malignancy": "None",
        "description": "The AI model has not detected patterns consistent with glioma, meningioma, or pituitary adenoma in the provided MRI scan. This result suggests normal brain parenchyma without obvious mass lesions. However, this is an AI-assisted preliminary screening and should not replace comprehensive radiological evaluation.",
        "symptoms": [
            "Absence of tumor-related symptoms is expected",
            "Headaches may have benign causes: tension, migraine, dehydration",
            "Dizziness can be vestibular or cardiovascular in origin",
            "Cognitive symptoms may be due to other medical conditions"
        ],
        "risk_factors": [
            "N/A - No tumor detected"
        ],
        "diagnosis": [
            "Complete MRI brain protocol with multiple sequences",
            "Clinical correlation with patient history and examination",
            "Follow-up imaging if symptoms persist or worsen",
            "Consider other diagnostic modalities if clinically indicated"
        ],
        "treatments": [
            "No tumor-specific treatment required",
            "Continue routine health maintenance and screening",
            "Address any symptoms with appropriate specialists",
            "Healthy lifestyle: exercise, balanced diet, adequate sleep",
            "Regular medical check-ups as recommended"
        ],
        "prognosis": "Excellent. No tumor-related health concerns identified. Continue normal health monitoring and consult healthcare provider for any new or persistent symptoms.",
        "prevalence": "The majority of brain MRI scans are normal or show age-appropriate changes without tumors. Brain tumors affect less than 1% of the population.",
        "color": "#00E676",
        "icon": "✅"
    }
}

# Load model
try:
    model = model
    print("✓ Model loaded successfully")
except Exception as e:
    print(f"✗ Model loading error: {e}")
    model = None

def preprocess_image(img):
    try:
        if img is None:
            return None
        img = img.convert("RGB").resize(IMG_SIZE, Image.Resampling.LANCZOS)
        arr = np.array(img, dtype=np.float32) / 255.0
        return np.expand_dims(arr, axis=0)
    except Exception as e:
        print(f"Preprocessing error: {e}")
        return None

def create_confidence_chart(predictions):
    colors = ["#FF1744", "#FF9100", "#00E676", "#9C27B0"]
    fig = go.Figure()
    for i, (name, prob) in enumerate(zip(CLASS_NAMES, predictions)):
        fig.add_trace(go.Bar(
            y=[name.capitalize()],
            x=[prob * 100],
            orientation='h',
            marker=dict(color=colors[i], line=dict(color='rgba(255,255,255,0.3)', width=2)),
            text=f"{prob*100:.2f}%",
            textposition='inside',
            textfont=dict(size=14, color='white', family='Arial Black'),
            width=0.5,
            hovertemplate=f'<b>{name.capitalize()}</b><br>Confidence: %{{x:.2f}}%<extra></extra>',
            showlegend=False
        ))
    fig.update_layout(
        title={"text": "Classification Confidence Levels", 'font': {'size': 22, 'color': '#00e5ff', 'family': 'Arial Black'}, 'x': 0.5, 'xanchor': 'center'},
        xaxis=dict(range=[0, 100], title="Confidence (%)", titlefont=dict(size=14, color='#aaa'), showgrid=True, gridcolor='rgba(255,255,255,0.1)', zeroline=False),
        yaxis=dict(showgrid=False, tickfont=dict(size=13, color='white')),
        height=350,
        margin=dict(l=120, r=40, t=80, b=60),
        paper_bgcolor="rgba(10,0,26,0.8)",
        plot_bgcolor="rgba(0,0,0,0)",
        hovermode='closest'
    )
    return fig

def create_probability_pie(predictions):
    colors = ["#FF1744", "#FF9100", "#00E676", "#9C27B0"]
    fig = go.Figure(data=[go.Pie(
        labels=[name.capitalize() for name in CLASS_NAMES],
        values=predictions * 100,
        hole=0.4,
        marker=dict(colors=colors, line=dict(color='#000', width=2)),
        textinfo='label+percent',
        textfont=dict(size=12, color='white'),
        hovertemplate='<b>%{label}</b><br>%{value:.2f}%<extra></extra>'
    )])
    fig.update_layout(
        title={"text": "Probability Distribution", 'font': {'size': 20, 'color': '#00e5ff'}, 'x': 0.5, 'xanchor': 'center'},
        height=350,
        margin=dict(l=20, r=20, t=60, b=20),
        paper_bgcolor="rgba(10,0,26,0.8)",
        showlegend=True,
        legend=dict(font=dict(color='white', size=11), bgcolor='rgba(0,0,0,0.3)')
    )
    return fig

def generate_detailed_report(label, confidence, predictions, timestamp):
    data = MEDICAL_DATA[label]

    # Reduced spacing between list items
    symptoms_html = "<br>".join([f"<li style='margin-bottom:4px;'>{s}</li>" for s in data['symptoms']])
    risks_html = "<br>".join([f"<li style='margin-bottom:4px;'>{r}</li>" for r in data['risk_factors']])
    diagnosis_html = "<br>".join([f"<li style='margin-bottom:4px;'>{d}</li>" for d in data['diagnosis']])
    treatments_html = "<br>".join([f"<li style='margin-bottom:4px;'>{t}</li>" for t in data['treatments']])

    if confidence > 90:
        conf_msg = "Very High Confidence"; conf_color = "#00ff00"
    elif confidence > 75:
        conf_msg = "High Confidence"; conf_color = "#7CFC00"
    elif confidence > 60:
        conf_msg = "Moderate Confidence"; conf_color = "#FFD700"
    else:
        conf_msg = "Low Confidence - Further Review Recommended"; conf_color = "#FF8C00"

    html = f"""
    <div style="background: linear-gradient(135deg, #0a001a 0%, #1a0033 100%);
                padding: 40px; border-radius: 25px;
                border: 3px solid {data['color']};
                box-shadow: 0 0 40px {data['color']}80, inset 0 0 20px rgba(0,0,0,0.5);
                color: white; font-family: 'Segoe UI', Arial, sans-serif;">
        <div style="text-align: center; margin-bottom: 30px;">
            <div style="font-size: 60px; margin-bottom: 10px;">{data['icon']}</div>
            <h1 style="color: {data['color']}; font-size: 44px; margin: 0; text-shadow: 0 0 20px {data['color']};">
                {data['title'].upper()}
            </h1>
            <p style="color: #888; font-size: 18px; margin-top: 5px;">{data['full_name']}</p>
        </div>
        <div style="background: rgba(0,0,0,0.4); padding: 25px; border-radius: 15px; margin-bottom: 30px; border-left: 6px solid {conf_color};">
            <div style="display: flex; justify-content: space-between; align-items: center; flex-wrap: wrap; gap: 20px;">
                <div>
                    <h2 style="color: {conf_color}; margin: 0; font-size: 32px;">{confidence:.2f}%</h2>
                    <p style="color: #aaa; margin: 5px 0 0 0; font-size: 14px;">{conf_msg}</p>
                </div>
                <div style="text-align: right;">
                    <p style="margin: 0; color: #00e5ff; font-size: 12px;">Analysis Timestamp</p>
                    <p style="margin: 5px 0 0 0; color: white; font-size: 14px;">{timestamp}</p>
                </div>
            </div>
        </div>
        <div style="background: rgba(0,0,0,0.3); padding: 15px; border-radius: 10px; margin-bottom: 25px;">
            <p style="margin: 0;"><strong style="color: #00ffaa;">Malignancy Level:</strong>
            <span style="color: {data['color']};">{data['malignancy']}</span></p>
        </div>
        <div style="margin-bottom: 25px;">
            <h3 style="color: {data['color']}; font-size: 24px; margin-bottom: 15px; border-bottom: 2px solid {data['color']}; padding-bottom: 10px;">📋 Overview</h3>
            <p style="line-height: 1.9; font-size: 15px; color: #ddd;">{data['description']}</p>
        </div>
        <div style="margin-bottom: 25px;">
            <h3 style="color: {data['color']}; font-size: 24px; margin-bottom: 15px; border-bottom: 2px solid {data['color']}; padding-bottom: 10px;">🔍 Common Symptoms</h3>
            <ul style="line-height: 1.5; font-size: 14px; color: #ddd; padding-left: 20px; margin-top: 8px;">{symptoms_html}</ul>
        </div>
        <div style="margin-bottom: 25px;">
            <h3 style="color: {data['color']}; font-size: 24px; margin-bottom: 15px; border-bottom: 2px solid {data['color']}; padding-bottom: 10px;">⚠️ Risk Factors</h3>
            <ul style="line-height: 1.5; font-size: 14px; color: #ddd; padding-left: 20px; margin-top: 8px;">{risks_html}</ul>
        </div>
        <div style="margin-bottom: 25px;">
            <h3 style="color: {data['color']}; font-size: 24px; margin-bottom: 15px; border-bottom: 2px solid {data['color']}; padding-bottom: 10px;">🔬 Diagnostic Evaluation</h3>
            <ul style="line-height: 1.5; font-size: 14px; color: #ddd; padding-left: 20px; margin-top: 8px;">{diagnosis_html}</ul>
        </div>
        <div style="margin-bottom: 25px;">
            <h3 style="color: {data['color']}; font-size: 24px; margin-bottom: 15px; border-bottom: 2px solid {data['color']}; padding-bottom: 10px;">💊 Treatment Approaches</h3>
            <ul style="line-height: 1.5; font-size: 14px; color: #ddd; padding-left: 20px; margin-top: 8px;">{treatments_html}</ul>
        </div>
        <div style="background: rgba(0,100,200,0.15); padding: 20px; border-radius: 12px; margin-bottom: 25px; border-left: 5px solid #00e5ff;">
            <h3 style="color: #00e5ff; font-size: 20px; margin-top: 0;">📊 Prognosis</h3>
            <p style="line-height: 1.8; font-size: 14px; color: #ddd; margin-bottom: 0;">{data['prognosis']}</p>
        </div>
        <div style="background: rgba(100,0,200,0.15); padding: 20px; border-radius: 12px; margin-bottom: 30px; border-left: 5px solid #9C27B0;">
            <h3 style="color: #9C27B0; font-size: 20px; margin-top: 0;">📈 Epidemiology</h3>
            <p style="line-height: 1.8; font-size: 14px; color: #ddd; margin-bottom: 0;">{data['prevalence']}</p>
        </div>
        <div style="background: linear-gradient(135deg, #300008 0%, #1a0000 100%); padding: 25px; border-radius: 15px; border: 3px solid #ff1744; box-shadow: 0 0 20px rgba(255,23,68,0.3);">
            <h3 style="color: #ff1744; margin-top: 0; font-size: 20px;">⚕️ IMPORTANT MEDICAL DISCLAIMER</h3>
            <p style="line-height: 1.8; font-size: 14px; margin-bottom: 15px;">
                <strong>This is NOT a medical diagnosis.</strong> This AI-powered analysis is a preliminary screening tool designed to assist healthcare professionals. It should never replace professional medical evaluation, diagnosis, or treatment.
            </p>
            <ul style="font-size: 13px; line-height: 1.7; color: #ffcccb; padding-left: 20px;">
                <li>Only board-certified radiologists and physicians can interpret MRI scans</li>
                <li>AI predictions may have false positives or false negatives</li>
                <li>Clinical correlation with complete medical history is essential</li>
                <li>Multiple MRI sequences and views are needed for accurate diagnosis</li>
                <li>Consult a qualified healthcare provider for proper evaluation</li>
            </ul>
        </div>
    </div>
    """
    return html

def predict_tumor(img_pil):
    if img_pil is None:
        return ("<div style='color: #ff1744; text-align: center; padding: 40px;'><h2>⚠️ No Image Uploaded</h2><p>Please upload an MRI scan to begin analysis.</p></div>", None, None, "")
    if model is None:
        return ("<div style='color: #ff1744; text-align: center; padding: 40px;'><h2>❌ Model Not Loaded</h2><p>The AI model could not be loaded. Please check the model path.</p></div>", None, None, "")
    try:
        preprocessed = preprocess_image(img_pil)
        if preprocessed is None:
            raise ValueError("Image preprocessing failed")
        predictions = model.predict(preprocessed, verbose=0)[0]
        confidence = float(np.max(predictions) * 100)
        predicted_idx = int(np.argmax(predictions))
        predicted_label = CLASS_NAMES[predicted_idx]
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        bar_chart = create_confidence_chart(predictions)
        pie_chart = create_probability_pie(predictions)
        report = generate_detailed_report(predicted_label, confidence, predictions, timestamp)
        summary = f"""
### 🎯 Quick Summary
**Prediction:** {predicted_label.capitalize()}
**Confidence:** {confidence:.2f}%
**Timestamp:** {timestamp}

**All Class Probabilities:**
- Glioma: {predictions[0]*100:.2f}%
- Meningioma: {predictions[1]*100:.2f}%
- No Tumor: {predictions[2]*100:.2f}%
- Pituitary: {predictions[3]*100:.2f}%
        """
        return report, bar_chart, pie_chart, summary
    except Exception as e:
        error_msg = f"""
        <div style='background: #1a0000; color: #ff1744; padding: 30px; border-radius: 15px; border: 2px solid #ff1744;'>
            <h2>❌ Analysis Error</h2>
            <p>An error occurred during analysis: {str(e)}</p>
            <p style='color: #aaa; font-size: 12px; margin-top: 20px;'>Please ensure you've uploaded a valid MRI scan image.</p>
        </div>
        """
        return error_msg, None, None, ""

# Custom CSS
custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Orbitron:wght@400;700;900&family=Exo+2:wght@300;400;600&display=swap');
body { background: linear-gradient(135deg, #0f0020 0%, #1a0033 50%, #0f0020 100%); background-attachment: fixed; }
.gradio-container { max-width: 1800px !important; background: rgba(10, 0, 26, 0.6) !important; backdrop-filter: blur(10px); border-radius: 30px; box-shadow: 0 0 50px rgba(0, 229, 255, 0.3); }
.gr-button-primary { background: linear-gradient(135deg, #00e5ff 0%, #0080ff 100%) !important; border: none !important; font-family: 'Orbitron', sans-serif !important; font-weight: 700 !important; font-size: 18px !important; padding: 15px 40px !important; border-radius: 50px !important; box-shadow: 0 0 30px rgba(0, 229, 255, 0.5) !important; transition: all 0.3s ease !important; }
.gr-button-primary:hover { transform: translateY(-3px) !important; box-shadow: 0 0 40px rgba(0, 229, 255, 0.8) !important; }
.gr-button-secondary { background: linear-gradient(135deg, #ff1744 0%, #ff5722 100%) !important; border: none !important; font-family: 'Orbitron', sans-serif !important; color: white !important; }
h1, h2, h3 { font-family: 'Orbitron', sans-serif !important; }
.gr-box { border-radius: 20px !important; background: rgba(0, 0, 0, 0.3) !important; border: 1px solid rgba(0, 229, 255, 0.3) !important; }
"""

# Build Gradio Interface
with gr.Blocks(title="NeuroVision AI - Brain MRI Scanner") as app:
    gr.HTML(f"<style>{custom_css}</style>")

    gr.HTML("""
    <div style="text-align: center; padding: 60px 20px 40px 20px;
                background: linear-gradient(135deg, rgba(0,229,255,0.1) 0%, rgba(255,23,68,0.1) 100%);
                border-radius: 25px; margin-bottom: 30px;
                box-shadow: 0 0 50px rgba(0,229,255,0.2);">
        <h1 style="font-family: 'Orbitron', sans-serif; font-size: 56px; color: #00e5ff; margin: 0;
                   text-shadow: 0 0 30px #00e5ff, 0 0 50px #0080ff;">
            NeuroVision AI
        </h1>
        <p style="font-family: 'Exo 2', sans-serif; font-size: 24px; color: #aaa; margin: 15px 0 0 0;">
            Advanced Deep Learning Brain Tumor Classification System
        </p>
        <p style="font-size: 16px; color: #888; margin-top: 10px; max-width: 900px; margin-left: auto; margin-right: auto;">
            Upload a brain MRI scan (T1-weighted with contrast recommended) for AI-powered preliminary screening of glioma, meningioma, pituitary adenoma, or no tumor.
        </p>
    </div>
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.HTML("""
            <div style="background: rgba(0,0,0,0.4); padding: 20px; border-radius: 15px;
                        border-left: 5px solid #00e5ff; margin-bottom: 20px;">
                <h3 style="color: #00e5ff; margin: 0 0 10px 0;">📤 Upload MRI Scan</h3>
                <p style="color: #ccc; font-size: 14px; margin: 0;">
                    Supported formats: JPG, PNG<br>
                    Recommended: Axial or coronal T1 post-contrast sequences
                </p>
            </div>
            """)
            input_image = gr.Image(type="pil", label="Upload Brain MRI", height=400)
            analyze_btn = gr.Button("🔬 Analyze Scan", variant="primary", size="lg")

        with gr.Column(scale=2):
            with gr.Tabs():
                with gr.Tab("📊 Detailed Medical Report", id="report"):
                    report_output = gr.HTML(label="Analysis Report")
                with gr.Tab("📈 Confidence Visualization"):
                    # Charts stacked vertically
                    bar_chart_output = gr.Plot(label="Confidence Bar Chart")
                    pie_chart_output = gr.Plot(label="Probability Distribution Pie")
                with gr.Tab("📋 Quick Summary"):
                    summary_output = gr.Markdown(label="Prediction Summary")

    analyze_btn.click(
        fn=predict_tumor,
        inputs=input_image,
        outputs=[report_output, bar_chart_output, pie_chart_output, summary_output]
    )

    gr.HTML("""
    <div style="margin-top: 50px; padding: 30px; background: rgba(255,23,68,0.15);
                border-radius: 20px; border: 2px dashed #ff1744; text-align: center;">
        <p style="color: #ff8a80; font-size: 18px; margin: 0;">
            ⚕️ <strong>This tool is for educational and research purposes only.</strong><br>
            NeuroVision AI is not FDA-approved and cannot provide medical diagnoses.<br>
            Always consult a qualified neuroradiologist or neurosurgeon for clinical decision-making.
        </p>
        <p style="color: #888; font-size: 13px; margin-top: 15px;">
            Model trained on public datasets • Accuracy depends on image quality and sequence type
        </p>
    </div>
    """)

# Launch the app
if __name__ == "__main__":
    app.launch(share=True)


✓ Model loaded successfully
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9f03bf3e3fa30886f9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
